In [1]:
import json
import sqlite3

Load scraped data
Most recent: 02/21/2022

In [2]:
# Path of scraped data
with open('em_scraped_data_02212022.json') as f:
    data = json.load(f)

Connect to sqlite3 database

In [4]:
con = sqlite3.connect('em_scraped_data_03122023_database.db')
cursor = con.cursor()

Create tables

In [5]:
sql_createProvider = '''CREATE TABLE Provider (
    EMID            INTEGER       PRIMARY KEY
                                  NOT NULL,
    Name            VARCHAR (64)  NOT NULL,
    State           VARCHAR (64)  NOT NULL,
    City            VARCHAR (64)  NOT NULL,
    PhoneNumber     VARCHAR (64),
    Link            VARCHAR (128),
    Age             VARCHAR (64),
    LastSeenOnline  VARCHAR (64),
    PersonalWebsite VARCHAR (128),
    Services        VARCHAR (128) 
);'''
cursor.execute(sql_createProvider)

In [6]:
sql_createReviews = ''' CREATE TABLE Reviews (
    ReviewID     INTEGER       PRIMARY KEY
                               NOT NULL,
    ReviewerID   INTEGER       NOT NULL
                               REFERENCES Reviewer (ID),
    Username     VARCHAR (64)  NOT NULL,
    ProviderEMID INTEGER       REFERENCES Provider (EMID) 
                               NOT NULL,
    IncallRates  VARCHAR (64),
    OutcallRates VARCHAR (64),
    DateAdded    VARCHAR (64),
    NameUsed     VARCHAR (64),
    State        VARCHAR (64),
    City         VARCHAR (64),
    PhoneNumber  VARCHAR (64),
    Tattoos      VARCHAR (64),
    PhotosReal   VARCHAR (64),
    Link         VARCHAR (128) 
);'''
cursor.execute(sql_createReviews)

In [7]:
sql_createReviewer = ''' CREATE TABLE Reviewer (
    ID       INTEGER      NOT NULL
                          PRIMARY KEY,
    Username VARCHAR (64) NOT NULL
);'''
cursor.execute(sql_createReviewer)

In [8]:
sql_createImages = '''CREATE TABLE Images (
    ProviderEMID  VARCHAR (64)  NOT NULL
                                REFERENCES Provider (EMID),
    ImageURL      VARCHAR (128) NOT NULL
                                PRIMARY KEY,
    BaseImageName VARCHAR (128),
    [phash8-1]    VARCHAR (64),
    [phash16-1]   VARCHAR (128),
    sha256        VARCHAR (128) 
);'''
cursor.execute(sql_createImages)

Add new providers, reviewers, reviews, and images to database

In [9]:
# Add provider to database
for provider in data.items():
    # Phone Numbers are sometimes not present in data, so this makes an empty entry to avoid errors
    if "Phone Number" not in provider[1]:
        provider[1]["Phone Number"] =""
    # last_seen_online sometimes has no entry, so this creates an empty one
    if "last_seen_online" not in provider[1]:
        provider[1]["last_seen_online"] =""
    # personal_website sometimes has no entry
    if provider[1]["personal_website"] is None:
        provider[1]["personal_website"] =""
    # Try to insert the provider in the table
    try:
        sqlite_insert_provider_query = ("\nINSERT INTO Provider (EMID, Name, State, City, PhoneNumber, Link, Age, LastSeenOnline, PersonalWebsite, Services) VALUES ('" +provider[1]["emid"]+ "','"+provider[1]["provider_name"]+"','"+provider[1]["State"]+"','"+provider[1]["City"]+"','"+provider[1]["Phone Number"]+"','"+provider[0]+"','"+provider[1]["Age"]+"','"+provider[1]["last_seen_online"]+"','"+provider[1]["personal_website"]+"','"+provider[1]["Services"]+"');")
        count = cursor.execute(sqlite_insert_provider_query)
    # Pass if there is an error
    except:
        #print("Provider did not work\n")
        pass
    # Insert each review for the provider
    for review in provider[1]["review_data"].items():
        # The next 4 lines grabs the unique review ID for each review
        # For data scraped 2/21/2022, the review ID is not in its own variable
        link = review[0]
        link_length = len(link)
        review_id = link[link_length - 6 : ]
        if (review_id[0] == '-'): review_id = review_id[1:]
        
        # If no tattoo data, create empty entry
        if "Tattoos" not in review[1]:
            review[1]["Tattoos"] =""
        # If no Photos Real data, create empty entry
        if "Photos Real" not in review[1]:
            review[1]["Photos Real"] =""
        # If no phone number, create empty entry
        if "Phone number" not in review[1]:
            review[1]["Phone number"] =""
        # If no incall_rates, create empty entry
        if review[1]["incall_rates"] == "not listed":
            incallRates = "not listed"
        # Create string if multiple incall_rates
        elif review[1]["incall_rates"] != "not listed":
            incallRates = ""
            for i in review[1]["incall_rates"]:
                incallRates += ''.join(i)
                incallRates += " "
        # If no outcall_rates, create empty entry
        if review[1]["outcall_rates"] == "not listed":
            outcallRates = "not listed"
        # Create string if multiple outcall rates
        elif review[1]["outcall_rates"] != "not listed":
            outcallRates = ""
            for i in review[1]["outcall_rates"]:
                outcallRates += ''.join(i)
                outcallRates += " "        
        # Insert review data
        try:
            sqlite_insert_reviews_query = ("\nINSERT INTO Reviews (ReviewID, ReviewerID, Username, ProviderEMID,IncallRates,OutcallRates, DateAdded, NameUsed, State, City, PhoneNumber, Tattoos, PhotosReal, Link) VALUES ('"+review_id + "','" +review[1]["reviewer_id"] +"','"+review[1]["review_username"]+"','"+provider[1]["emid"]+"','"+incallRates+"','"+outcallRates+"','"+review[1]["date_added"]+"','"+review[1]["Named Used"]+"','"+review[1]["State"]+"','"+review[1]["City"]+"','"+review[1]["Phone number"]+"','"+review[1]["Tattoos"]+"','"+review[1]["Photos Real"]+"','"+review[0]+"');")
            count = cursor.execute(sqlite_insert_reviews_query)
        # Pass if fail
        except:
            pass
        
        # Insert reviewer data
        try:
            sqlite_insert_reviewer_query = ("\nINSERT INTO Reviewer (ID, Username) VALUES ('" +review[1]["reviewer_id"] +"','"+review[1]["review_username"]+"');")
            count = cursor.execute(sqlite_insert_reviewer_query)
        # Pass if fail
        except:
            pass
        
    # Insert image data
    for image in provider[1]["img_data"].items():
        try:
            sql_insert_image_query = ("\nINSERT INTO Images (ProviderEMID, ImageURL, BaseImageName, [phash8-1], [phash16-1], sha256) VALUES ('"+provider[1]["emid"]+"','"+image[1]["image_url"]+"','"+image[1]["base_image_name"]+"','"+image[1]["phash8-1"]+"','"+image[1]["phash16-1"]+"','"+image[1]["sha256"]+"');")
            count = cursor.execute(sql_insert_image_query)
        # Pass if fail
        except:
            pass
con.commit()
con.close()